## CU Boulder 4604/5604

#### September 28, 2020

####  Logistic regression revisited

### Logistic Regression revisited

- Last week we learned about logistic regression and regularization
- Today, we are going modify the notebook from last week in a few ways 

##### New stuff:
1. I filled in the missing functions from last week if you are curious. Everyone who submitted last week got full credit. These notebooks are a way to get your hands dirty with ML code so you learn the materials more deeply. They are not a way for me to quiz you on what you know. That is what HW, quizzes, exams are for.
2. We are going to add a regularization term to the loss function
3. We will introduce stochastic gradient descent. The well-known ML researcher [Dave Blei](http://www.cs.columbia.edu/~blei/) says stoachstic gradient descent is like walking from New York to Los Angeles by asking one person at a time for directions. And every person you ask for directions is drunk. Even though this seems like a bad way to get across the country, we will see that is has clear advantages
4. I changed the code from gradient ascent to gradient descent (flipping signs as needed). Recall that maximizing log likelihood is the same as minimizing negative log likelihood.

### Annoucements 

- HW2 is out
- This notebook is pretty similar to HW2
- In the real world, it is OK to lean on sklearn's implementation of logistic regression
- But taking this class gives you a chance to actually understand how it works. This notebook should make the sklearn code much less mysterious

In [9]:
%matplotlib inline

import math
import matplotlib.pyplot as plt
import numpy as np
import random
import pandas as pd
np.random.seed(42)

In [8]:
def logistic(z):
    return 1/(1 + np.exp(-z))


def log_prob(z, y_i):
    '''
    Returns the log_prob for one point
    '''
    fz = logistic(z)
    return y_i * np.log(fz) + (1 - y_i) * np.log(1 - fz)


def neg_log_likelihood(X, w, y):
    '''Compute the negative log likelihood'''
    L = 0
    for _x,_y in zip(X, y):
        z = w.dot(_x)
        L += log_prob(z=z, y_i=_y)
    return -1 * L


def fast_logistic(X, w):
    '''Compute the logistic function over many data points'''
    return 1/(1 + np.exp(-1 * X.dot(w)))


def grad(_X, w, _y, lambda_=.5):
    '''
    Return the gradient
    
    - https://web.stanford.edu/~jurafsky/slp3/5.pdf
    '''
    grad = np.zeros_like(w)
    
    N,D= _X.shape
    
    b = _X * (fast_logistic(_X, w) - _y).reshape((N, 1))

    return np.sum(b, axis=0) + (lambda_ * 2 * w)


def squared_l2_norm(w):
    '''
    $\sqrt{\Sigma x_i^2} ^ 2
    
    '''
    # implement this one
    return sum(w ** 2)


def grad_decent(_X, _y, eta = .0001, lambda_ = 0, tolerance=1e-4, verbose=True, batch_size=None, iters=None):
    '''
    - Perform gradient ascent
    - This function is basically the same as in the Adeline notebook
    - Of course, the gradient is different, because it is a different function
    '''
    w = np.random.uniform(low=-5, high=2, size=dim_)
    last = 0
    losses = []
    for i in range(1000):
        if i > iters and iters is not None:
            break
        this_ll = neg_log_likelihood(_X, w, _y)
        loss = this_ll + lambda_ * squared_l2_norm(w)
        losses.append(loss)
        if verbose:
            print("iter: {}, loss: {}, accuracy: {}".format(i, loss, accuracy(_X, w, _y)))
        if(abs(this_ll - last) < tolerance): break
        last = this_ll
        
        if batch_size is None:
            w -= eta * grad(_X, w, _y, lambda_=lambda_)
        else:
            _N,F = _X.shape
            idx = np.random.randint(_N, size=batch_size)
            w -= eta * grad(_X[idx], w, _y[idx], lambda_=lambda_)/batch_size
        
    return w, losses

def prediction(X, w, threshold=.5):
    '''
    - Return a Boolean array of length N.
    - The array should be True if the weights dotted with the features for a given instance is greater than .5
    '''
    N, D = X.shape
    return X.dot(w) > threshold

def accuracy(X, w, y):
    '''
    Return a value between 0 and 1, showing the fraction of data points which have been classified correctly
    '''
    return np.mean(prediction(X, w) == y)

def init_data(N, dim_):
    '''
    Initialize data. Note how we generate y below. We know how the data is generated.
    '''
    w = np.random.uniform(low=-1, high=1, size=dim_)
    X = (np.random.rand(dim_ * N) > .5).astype(int)
    X = X.reshape(N, dim_)

    z_ = X.dot(w) + np.random.uniform(low=-1, high=1, size=X.dot(w).size)

    y =  1/(1 + np.exp(-1 * z_)) > .5
    
    return X, y

np.random.seed(42)

N = 10000
dim_ = 10

w = np.random.uniform(low=-5, high=2, size=dim_)

X, y = init_data(N, dim_)

split = int(N/2)

X_train = X[0:split]
X_test = X[split:]
y_train = y[0:split]
y_test = y[split:]


lambda_ = .1
w, losses = grad_decent(X_train, y_train, eta=0.01, tolerance=.0001, iters=100, verbose=True, lambda_=lambda_, batch_size= 50)


iter: 0, loss: 5793.811507862259, accuracy: 0.8196
iter: 1, loss: 5790.394373029358, accuracy: 0.8196
iter: 2, loss: 5786.852617700935, accuracy: 0.8196
iter: 3, loss: 5782.6129072522435, accuracy: 0.8196
iter: 4, loss: 5780.136026044902, accuracy: 0.8196
iter: 5, loss: 5777.921271115245, accuracy: 0.8196
iter: 6, loss: 5775.960619615353, accuracy: 0.8196
iter: 7, loss: 5773.633381017308, accuracy: 0.8196
iter: 8, loss: 5771.149291397594, accuracy: 0.8196
iter: 9, loss: 5767.698772255626, accuracy: 0.8196
iter: 10, loss: 5764.169855371659, accuracy: 0.8196
iter: 11, loss: 5762.288864998929, accuracy: 0.8196
iter: 12, loss: 5756.713485871317, accuracy: 0.8196
iter: 13, loss: 5753.129485598977, accuracy: 0.8196
iter: 14, loss: 5749.279896535418, accuracy: 0.8196
iter: 15, loss: 5746.273395537522, accuracy: 0.8196
iter: 16, loss: 5740.713012129488, accuracy: 0.8196
iter: 17, loss: 5735.8874671459, accuracy: 0.8196
iter: 18, loss: 5733.342031766168, accuracy: 0.8196
iter: 19, loss: 5730.16

### Questions: normalization
- Complete the L2 norm function
- What does the variable `lambda` do in the code above? 
 - It penalizes for large weights to prevent overfitting
- What happens if you set `lambda` to a huge number? What happens if you set `lambda` to a small number?  What should you see in terms of accuracy and the norm of the weights? Try systematically varying lambda
 - The size of lambda determines how large the penalty will be for large weights. The smaller lambda is, the smaller the penalty for the weights.
 

### Stochastic gradient descent
- Computing the gradient can be inefficient. Why? 
 - It can be costly with large n beacuse gradient descent factors in all instances when updating weights
- What if instead of computing the whole gradient, we took a sample of the gradient
- This is the idea of "stochastic" gradient descent

### Questions: Stochastic gradient descent
- How noisy is the optimization if you vary the loss
- Print the loss and vary the batch size
- How do you think that varying eta will vary the noise 
 - The noise will not vary much because our stepsize (eta) is small
- How do you think that varying batch size will vary noise
 - a larger batch size should result in a smaller loss